In [ ]:
# import libraries
%matplotlib inline

from itertools import combinations_with_replacement
from functools import partial

from math import *
import matplotlib.pylab as plt
import numpy as np
from numpy.linalg import pinv

import pandas as pd

from scipy.optimize import minimize, NonlinearConstraint, show_options

# from optimize._constraints import NonlinearConstraint, LinearConstraint

# from optimize._trustregion_constr import _minimize_trustregion_constr as minimize
# from optimize._minimize import minimize

In [ ]:
data_file = 'panel_data.xlsx'
# data_file = 'bottle_data.xlsx'
# data_file = 'brake_rotor_data.xlsx'
num_pareto_points = 100

In [ ]:
df = pd.read_excel(data_file,header=(0,1,2,3),index_col=0)

data = df.to_numpy()

input_data = df.xs('input',axis=1,level='type')
output_data = df.xs('output',axis=1,level='type')


In [ ]:
df

In [ ]:
# define input data features
inputs = input_data.to_numpy()

# define the target values
outputs = output_data.to_numpy()

In [ ]:
inputs, outputs

In [ ]:
# create model

def get_response_surface(inputs, outputs):

    response_surfaces = []

    terms = list(combinations_with_replacement(range(inputs.shape[1]), 1))
    terms.extend(combinations_with_replacement(range(inputs.shape[1]), 2))  
  
    for current_output in range(outputs.shape[1]):
        A = np.ones( (inputs.shape[0], len(terms) + 1) )
        rhs = np.zeros( inputs.shape[0] )

        for i,row in enumerate(outputs):
            rhs[i] = row[current_output]
            for j,term in enumerate(terms):
                A[i,j+1] = inputs[i].take(term).prod()

        response_surfaces.append(pinv(A)@rhs)

    # first coefficient is the constant coefficient
    return terms, response_surfaces


def evaluate_response_surface(term_indices_list, rs_coefficients, x, factor=1.0, offset=0.0):
    terms = np.ones(len(term_indices_list)+1)

    for i, term_indices in enumerate(term_indices_list):
        terms[i+1] = x.take(term_indices).prod()

    return factor*(terms.dot(rs_coefficients) + offset)


def evaluate_response_surface_grad(term_indices_list, rs_coefficients, x, factor=1.0, offset=0.0):

    grad_terms = np.zeros( (len(x), len(term_indices_list)+1) )

    for i, term_indices in enumerate(term_indices_list):
        for j in range(len(x)):
            if len(term_indices) == 1 and term_indices[0] == j:
                grad_terms[j, i+1] = 1
            if len(term_indices) == 2:
                if term_indices[0] == j and term_indices[1] == j:
                    grad_terms[j, i+1] = 2.0*x[j]
                elif term_indices[0] == j:
                    grad_terms[j, i+1] = x[term_indices[1]]
                elif term_indices[1] == j:
                    grad_terms[j, i+1] = x[term_indices[0]]


    return factor*grad_terms.dot(rs_coefficients)

terms, response_surfaces = get_response_surface(inputs, outputs)
response_surfaces

In [ ]:
# determine pareto axes and axes goals
y_axis_df = output_data.xs('y',axis=1,level='pareto axis')
x_axis_df = output_data.xs('x',axis=1,level='pareto axis')

y_axis_label = y_axis_df.columns.get_level_values(0).values[0]
y_axis_index = output_data.columns.get_level_values(0).get_loc(y_axis_label)
y_axis_goal = y_axis_df.columns.get_level_values(1)

x_axis_label = x_axis_df.columns.get_level_values(0).values[0]
x_axis_index = output_data.columns.get_level_values(0).get_loc(x_axis_label)
x_axis_goal = x_axis_df.columns.get_level_values(1)


In [ ]:
# setup the bounds constraints for each input (use input data ranges)
# also, add any equality constraints that have been specified for the input values by setting
# the upper and lower bounds to be equal

bounds = []

for index,column in enumerate(input_data):
    if type(column[1]) is float or type(column[1]) is int:
        bounds.append((column[1], column[1]))
    
    else:
        # inequality constraint
    
        lower_lim = min(input_data[column])
        upper_lim = max(input_data[column])
        
        bounds.append((lower_lim, upper_lim))
    

In [ ]:
constraints = []

# add equality constraints for any that are specified for the outputs
for index,column in enumerate(output_data):
    if type(column[1]) is float or type(column[1]) is int:
        print(index)
        constraints.append(NonlinearConstraint(partial(evaluate_response_surface, terms, response_surfaces[index]), 
                                               column[1], column[1],
                                               jac=partial(evaluate_response_surface_grad, terms, 
                                                           response_surfaces[index])))

In [ ]:
def objective_func(x, index, sign=1.0): 
    return evaluate_response_surface(terms, response_surfaces[index], x, factor=sign)

def objective_func_grad(x, index, sign=1.0): 
    return evaluate_response_surface_grad(terms, response_surfaces[index], x, factor=sign)

x_starting_point = input_data.median().to_numpy()

# first, get the x pareto value lower and upper limits
#x_max_starting_point = X[y[:,x_axis_index].argmax(),:]
res = minimize(objective_func, x_starting_point, args=(x_axis_index,-1.0), bounds=bounds, 
               constraints=constraints, method='trust-constr', options={'disp': False})
x_max = -res.fun

#x_min_starting_point = X[y[:,x_axis_index].argmin(),:]
res = minimize(objective_func, x_starting_point, args=(x_axis_index,1.0), bounds=bounds,
               constraints=constraints, method='trust-constr', options={'disp': False})
x_min = res.fun
#pareto_starting_x = res.x

pareto_points = np.linspace(x_min,x_max,num=num_pareto_points)



In [ ]:
pareto_input_values = []

y_axis_sign = 1 if y_axis_goal == 'min' else -1

for x_value in pareto_points:
    if x_axis_goal == 'min':
        limits = (-np.inf, x_value)
    else:
        limits = (x_value, np.inf)
        
    current_constraint = NonlinearConstraint(partial(evaluate_response_surface, terms, 
                                                     response_surfaces[x_axis_index]), 
                                             *limits,
                                             jac=partial(evaluate_response_surface_grad, terms, 
                                                         response_surfaces[x_axis_index]))
    
    res = minimize(objective_func, x_starting_point, args=(y_axis_index,y_axis_sign), bounds=bounds,
                   constraints=constraints+[current_constraint,], 
                   jac=objective_func_grad, method='trust-constr', options={'disp': False})

    #pareto_starting_x = res.x
    
    pareto_input_values.append(res.x)
        
    
    

In [ ]:
pareto_output_values = []
for current_input in pareto_input_values:
    current_output = []
    for i in range(outputs.shape[1]):
        current_output.append(evaluate_response_surface(terms, response_surfaces[i], current_input))

    pareto_output_values.append(current_output)
    
pareto_output_values = np.array(pareto_output_values)

In [ ]:
fig = plt.figure()

# plot original DOE values as points
plt.plot(outputs[:,x_axis_index],outputs[:,y_axis_index],'.')
plt.plot(pareto_output_values[:,x_axis_index],pareto_output_values[:,y_axis_index])
plt.xlabel(x_axis_label)
plt.ylabel(y_axis_label)